In [1]:
!pip install pytorch-tabnet

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings(action='ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/WASSUP/project/train_modified.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/WASSUP/project/test.csv')

In [5]:
train['Race'] = train['Race'].map({'Asian' : 0, 'Hispanic' : 0, 'Black' : 1, 'White' : 0})
test['Race'] = test['Race'].map({'Asian' : 0, 'Hispanic' : 0, 'Black' : 1, 'White' : 0})

In [6]:
# train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/WASSUP/project/train_modified.csv', header=None)
target = 'Diagnosis'    # 41번째 컬럼
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2,], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
# test_indices = train[train.Set=="test"].index

In [7]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
# unused_feat = ['Set']
unused_feat = ['Set','Income','Passive','Sleep','Appetite']
features = [ col for col in train.columns if col not in unused_feat+[target]]
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]



X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

# X_test = train[features].values[test_indices]
# y_test = train[target].values[test_indices]

Diagnosis 2
Sex 2
Race 2
Housing 2
Delay 2
Set 2


In [8]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=2, #10
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":30,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                    #    mask_type='sparsemax'
                       mask_type='entmax'
                      )

In [9]:
max_epochs = 400

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
)

epoch 0  | loss: 0.70026 | train_auc: 0.51825 | valid_auc: 0.47013 |  0:00:02s
epoch 1  | loss: 0.3235  | train_auc: 0.59138 | valid_auc: 0.60994 |  0:00:04s
epoch 2  | loss: 0.19107 | train_auc: 0.60113 | valid_auc: 0.58108 |  0:00:05s
epoch 3  | loss: 0.12361 | train_auc: 0.61887 | valid_auc: 0.60832 |  0:00:06s
epoch 4  | loss: 0.08705 | train_auc: 0.53278 | valid_auc: 0.52447 |  0:00:08s
epoch 5  | loss: 0.07349 | train_auc: 0.50452 | valid_auc: 0.49971 |  0:00:09s
epoch 6  | loss: 0.05926 | train_auc: 0.55281 | valid_auc: 0.53471 |  0:00:11s
epoch 7  | loss: 0.05569 | train_auc: 0.63493 | valid_auc: 0.61671 |  0:00:12s
epoch 8  | loss: 0.05164 | train_auc: 0.70866 | valid_auc: 0.69592 |  0:00:13s
epoch 9  | loss: 0.05095 | train_auc: 0.77057 | valid_auc: 0.74863 |  0:00:16s
epoch 10 | loss: 0.04847 | train_auc: 0.83272 | valid_auc: 0.82275 |  0:00:18s
epoch 11 | loss: 0.04552 | train_auc: 0.88147 | valid_auc: 0.87629 |  0:00:20s
epoch 12 | loss: 0.0465  | train_auc: 0.92698 | vali

In [10]:
X_train

array([[1.        , 0.        , 0.        , ..., 4.29210182, 6.76365441,
        1.16504887],
       [0.        , 1.        , 1.        , ..., 5.36433134, 6.68985993,
        3.03068498],
       [0.        , 1.        , 0.        , ..., 6.48908517, 6.74963239,
        1.5489868 ],
       ...,
       [0.        , 0.        , 0.        , ..., 3.05138744, 1.91050571,
        1.77369286],
       [0.        , 0.        , 1.        , ..., 3.0990486 , 6.11730503,
        2.98077843],
       [1.        , 1.        , 0.        , ..., 6.1250315 , 5.37804182,
        4.21148269]])

In [11]:
target = 'Diagnosis'    # 41번째 컬럼
if "Set" not in test.columns:
    test["Set"] = np.random.choice(["test"], p =[1], size=(test.shape[0],))

test_indices = test[test.Set=="test"].index

In [12]:
nunique = test.nunique()
types = test.dtypes

categorical_columns = []
categorical_dims =  {}
for col in test.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, test[col].nunique())
        l_enc = LabelEncoder()
        test[col] = test[col].fillna("VV_likely")
        test[col] = l_enc.fit_transform(test[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        test.fillna(test.loc[test_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
# unused_feat = ['Set']
unused_feat = ['Set','Income','Passive','Sleep','Appetite']
features = [ col for col in test.columns if col not in unused_feat+[target]]
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

Diagnosis 2
Sex 2
Race 2
Housing 2
Delay 2
Set 1


In [13]:
X_test = test[features].values[test_indices]
y_test = test[target].values[test_indices]

In [14]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)

In [15]:
test_auc

0.9968586949342845

In [16]:
preds = clf.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy=accuracy_score(y_test, preds)

In [19]:
accuracy

0.987